# CYPLAN255
### Urban Informatics and Visualization

# Lecture 09 -- Data Analysis
*******
February 14, 2024

<img src="https://upload.wikimedia.org/wikipedia/commons/7/70/Eug%C3%A8ne_Delacroix_-_The_Barque_of_Dante.jpg" width=80% title="The Barque of Dante, Eugène Delacroix (1822)">


# Agenda
1. Announcements
2. Review of last session
3. Data Analysis
4. For next time
5. Questions


# 1. Announcements

1. Assignment 2 due Monday
2. Assignment 1 will be graded soon

# 2. Review from Intro to Data Analysis

## 2.1 Indexing and Selecting

- use `[<row>, <column>]` indexing with `loc` (labels) and `iloc` (positions)
- use `df[<col>]` to select a column/Series from a DataFrame by column label
- use `df[[<col1>, <col2>]]` to select a subset of a DataFrame by column label

## 2.2 Merge Methods

Most of the below is taken directly from the pandas [docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#brief-primer-on-merge-methods-relational-algebra).

There are 4 main types of merges, but all of them require **four** arguments:
- `left`: first `DataFrame` or `Series` to merge
- `right`: second `DataFrame` or `Series` to merge with the first 
- `on`: the name of the "key" column(s) you'll use to identify matching/corresponding rows in each table
- `how`: the type of merge to perform (default is "inner")

Let's see how this works in practice. First we'll create some data:

In [ ]:
import pandas as pd

left = pd.DataFrame(

    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }

)

right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

And now we'll do some merges

**inner** join
![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_multiple.png)

In [ ]:
pd.merge(left, right, on=["key1", "key2"])  # inner is default "how", so we don't need to specify it

**left** join
![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_left.png)

In [ ]:
pd.merge(left, right, how="left", on=["key1", "key2"])

**right** join
![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_right.png)

In [ ]:
pd.merge(right, left, how="left", on=["key1", "key2"])

**outer** (A.K.A. full) join
![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_outer.png)

In [ ]:
pd.merge(left, right, how="outer", on=["key1", "key2"])

**cross** join (A.K.A. Cartesian Product)
![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_cross.png)

In [ ]:
pd.merge(left, right, how="cross").head()

## 2.3 Handling duplicate column names

In [ ]:
left = pd.DataFrame({"A": [4, 5], "B": [2, 2], "C": [5, 7]})
right = pd.DataFrame({"A": [4, 5, 6], "B": [2, 2, 2], "D": [9, 8, 1]})
pd.merge(left, right, on=["B"], how="inner")

### 2.3.1 Define your own suffixes!

**Keep all columns, but give them meaningful names**

In [ ]:
pd.merge(left, right, on="B", how="inner", suffixes=('_left', '_right'))

**Use suffixes to drop duplicate columns after merge**

In [ ]:
result = pd.merge(left, right, on="B", how="inner", suffixes=('_trash', ''))

In [ ]:
# keep only cols you want
good_cols = [col for col in result.columns if not 'trash' in col]
result[["B", "C", "A", "D"]]

In [ ]:
# drop cols you don't want
trash_cols = [col for col in result.columns if 'trash' in col]
result.drop(trash_cols, axis=1)  # "axis" tells pandas to drop columns (axis=1) or rows (axis=2)

### 2.3.2 Filter out duplicate columns before you merge!

In [ ]:
key = "B"
left_col_mask = [
    col for col in left.columns if (col not in right.columns) or (col == key)]
pd.merge(left[left_col_mask], right, on=key, how="inner")

# 3. Exploratory Data Analysis with pandas

**READ THE DOCS**

Most of the pandas-related material we saw **yesterday** corresponds to three sections of the pandas user guide:
1. [Basics](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html)
2. [Indexing and Selecting](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)
3. [Merge, join, concatenate, and compare](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)


Most of the material we're covering **today** is also covered in the following three sections of the pandas user guide:
1. [Group by: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)
2. [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)
3. [Chart visualization](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)

**Everything** we're covering in pandas is summarized nicely in the [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) section.

## 3.1 The `groupby()` method

For the first part of this lesson, we're going to be analyzing some Bay Area Census data which I've already grabbed for you (specifically the Summary File 1 (SF1) data from the 2010 Census). Refer to the [SF1 Data Dictionary](https://www.census.gov/prod/cen2010/doc/sf1.pdf) to see the list of column codes. I've compiled this data into an HDF5 file, which is kind of like a .zip archive that lets you unzip one file at a time.

In [ ]:
sf1 = pd.read_hdf('data/bay_sf1_small.h5', 'sf1_extract')
sf1.head()

**Note on HDF5 dependencies**

pandas uses the `pytables` library to interact with .h5 files. If you don't have it installed and you try to run the cell above, Python may complain. Fear not. Did you know you can execute bash terminal commands directly from a notebook?

In [ ]:
# !conda install pytables

### 3.1.1 Basic Data Transformations

In the world of machine learning, data transformation is sometimes called **feature extraction**. Both terms refer to the process of taking "raw" input data and manipulating it to create new, useful data. In the example below, we convert population totals to percentages:

In [ ]:
sf1['pct_black'] = sf1['P0030003'] / sf1['P0030001'] * 100
sf1['pct_asian'] = sf1['P0030005'] / sf1['P0030001'] * 100
sf1['pct_white'] = sf1['P0030002'] / sf1['P0030001'] * 100
sf1['pct_hisp'] = sf1['P0040003'] / sf1['P0040001'] * 100
sf1

Now add colums with percentage rental and population per square mile.

In [ ]:
sf1['pct_rent'] = sf1['H0040004'] / sf1['H0040001'] * 100
sf1['pop_sqmi'] = (sf1['P0010001'] / (sf1['arealand'] / 2589988))
sf1 = sf1[sf1['P0030001'] > 0]
sf1.sample(3)

Notice how when we create new columns they get automatically appended to the end (right) of the table

Now let's add county names to the dataframe so we get more readable output. First we'll create a dictionary to map the FIPS codes to county names

In [ ]:
county_fips_to_name = {
    '001': 'Alameda', '013': 'Contra Costa', '041': 'Marin', '055': 'Napa', '075': 'San Francisco',
    '081': 'San Mateo', '085': 'Santa Clara', '095': 'Solano', '097': 'Sonoma'}

Then we pass that dictionary to the `pd.Series.replace()` method to perform the conversion.

We could use the same assignment-based approach to create our new column:

In [ ]:
sf1['county_name'] = sf1['county'].replace(county_fips_to_name)

or we can use `pd.DataFrame.insert()` to tell pandas exactly where to stick the new column

In [ ]:
del sf1['county_name']  # drop the column we just created
sf1.insert(4, 'county_name', sf1['county'].replace(county_fips_to_name))
sf1.head(3)

Notice how `insert()` operates "in-place"?

### 3.1.2 Split-Apply-Combine

Groupby is a powerful method in pandas that follows the split-apply-combine approach to data manipulation.

<center><img src="https://wesmckinney.com/book/images/pda3_1001.png" width=40%></center>

**SPLIT**

Let's apply this approach to computing total population in each county in our dataset. First we create a groupby object, using county codes to group all the census blocks in sf1 into groups that share the same county code. This represents the **split** part of the workflow in the figure above

In [ ]:
grouped = sf1[['P0010001', 'county_name']].groupby('county_name')
grouped

**APPLY**

Now were ready to apply an operation to each group we've split. We call these **aggregation** methods because for each group they will take a series of values and combine them to produce one value, like a min/max/mean. pandas provides a bunch of built-in aggregation functions for use with `groupby` object. Some of the most common ones include:

* `count`
* `sum`
* `mean`
* `median`
* `std`, `var`
* `min`, `max`
* `idxmax`, `idxmin`
* `first`, `last`
* `quantile`

But you can also define and apply your own functions to use for aggregation.

**COMBINE**

To apply your chosen aggregation, you can call it directly as a method of your `groupby` object. The object pandas returns will be the **combined** outputs of this method for each of your groups:

In [ ]:
grouped.sum()

**SPLIT-APPLY-COMBINE**

Doing this in two steps like above is really just to clarify the two parts of the split and apply process that happen within a groupy operation. Normally we would not bother separately creating a groupby object -- we would just do this in one line:

In [ ]:
county_pop = sf1[['county_name', 'P0010001']].rename(
    columns={'P0010001': 'total_pop'}).groupby(
    'county_name').sum()
county_pop

## 3.2 Aggregating on multiple columns

Let's merge the county totals with the original sf1 data

In [ ]:
sf2 = pd.merge(sf1, county_pop, left_on='county_name', right_index=True, how='inner')
sf2.head()

Let's say we wanted to compute the population per square mile by county.  We could go ahead and create another dataframe with total area by county than then divide the total population by total area.

In [ ]:
county_land = sf1[['county_name', 'arealand']].groupby(sf1['county_name']).sum()
county_land

In [ ]:
sq_m_to_sq_mi = 2589988.11 
county_pop['total_pop'] / county_land['arealand'] * sq_m_to_sq_mi

Or, we could have done both aggregations at the same time!

In [ ]:
sf1[['county_name', 'P0010001', 'arealand']].groupby('county_name').sum()
# county_totals['pop_density'] = county_totals['P0010001'] / county_totals['arealand'] * sq_m_to_sq_mi
# county_totals

What if we want to apply different aggregations to different columns? 

In [ ]:
sf1.groupby('county_name').agg({'pct_asian': 'mean', 'P0010001': 'sum'})

### Question 1

Sometimes I write `df[<list of columns>].groupby()`, but sometimes I just do `df.groupby()`. Why?

## 3.3 Exercises:

Count the number of census blocks per county.

Calculate total households per county.

Calculate percent renters by county. (Careful not to calculate the mean percent rental across blocks in a county)

Calculate percent vacant by county.

Calculate mean, min and max pop_sqmi (at the block level) by county.

Calculate the 90th percentile of pop_sqmi (at the block level) by county.

In [ ]:
print('Number of census blocks per county:')

In [ ]:
print('Total households per county')

In [ ]:
print('percent renters by county')

In [ ]:
print('Percent vacant by county')

In [ ]:
print('Min, Max and Mean Population per SQMI by Census Block')

In [ ]:
print('90th Percentile of Population per SQMI at block level by County')

## 3.4 Cleaning Messy Data -- Craigslist Rental Listings

### 3.4.1 Loading data
Let's load some rental listings I scraped from Craigslist.  

In [ ]:
df = pd.read_csv('data/bay.csv')
df.head()

It turns out to be pretty messy. What problems do you see?


- Neighborhood names are in parentheses...
- Bedrooms and Square Feet are embedded in a single string in the bedrooms column along with other text...
- Price is formatted as a string with a dollar sign...
- Date is a string in a non-standard format...

So how can we go about cleaning these data up to use them for analysis?

Let's start with cleaning up the Price and Neighborhood variables.

### 3.4.2 String Ops

In [ ]:
df['price'] = df['price'].str.strip('$').astype('float64')
df['neighborhood'] = df['neighborhood'].str.strip().str.strip('(').str.strip(')')
df.head()

OK, now lets create Year, Month and Day columns

In [ ]:
df['month'] = df['date'].str.split().str[0]
df['day'] = df['date'].str.split().str[1].astype('int32')
df['year'] = df['date'].str.split().str[2].astype('int32')
df.head()

### 3.4.3 Datetime ops

pandas has special functions for dealing with `datetime` data types which make it much easier to do what we just did above. First we have to convert our date-like column to a `datetime` column

In [ ]:
df['date'] = pd.to_datetime(df['date'])

Now we can use the `dt` method (just like `.str.` for string ops) to get month, day, year, and whatever else we want:

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()

In [ ]:
df[['date', 'day', 'month', 'year', 'day_of_week']].head()

### 3.4.4 Complex string processing

Let's see how we might extract the bedrooms...

In [ ]:
df.beds_sqft.sample(5)

In [ ]:
def clean_bdrm(value):

    if isinstance(value, str):
        end = value.find('br')

        if end == -1:
            return
        
        else:
            start = value.find('/') + 2
            return int(value[start:end])

    else:
        return

df['bedrooms'] = df['beds_sqft'].map(clean_bdrm)

In [ ]:
df[['bedrooms', 'beds_sqft']]

And the same approach might work for creating a sqft column.

In [ ]:
def clean_sqft(value):

    if isinstance(value, str):
        end = value.find('ft')
        
        if end == -1:
            return

        else:
            if value.find('br') == -1:
                start = value.find('/') + 2
            else:
                start = value.find('-') + 2

            return int(value[start:end])

    else:
        return

In [ ]:
df['sqft'] = df['beds_sqft'].map(clean_sqft)
df.head()

## 3.5 Summarizing your data

Let's have a look at a statistical profile of our data

In [ ]:
df.describe()

- Why are there different counts on the columns?
- How do the price (rent) variable ranges look?  1 dollar in rent as min?  35,000 in rent as  maximum?
- What about sqft?  1 sqft min and 12,700 sqft max?
- You are now in the realm of real-world data, with **outliers**!

## 3.6 Dealing with outliers
When we talk about **outliers**, we're not talking about the Malcom Gladwell kind. The kind of outliers we're talking about are the kind that are so far-fetched that they more likely represent bad data than real observations. And even if they are real, they're so amazingly rare that we don't want them to bias our analysis. In either case, we need to get rid of them.

In the case of our Craigslist listings, we'll do this in three steps:
1. Find outliers in rent, say the top and bottom 1%
1. Analyze the data without missing data
1. Create a dataset that removes the outliers

### 3.6.1 Price

Let's get a quantile value at the 1st percentile to see the value that the top one percent of our records exceed

In [ ]:
low = df['price'].dropna().quantile(.01)
print(low)

And now the top 1%

In [ ]:
high = df['price'].dropna().quantile(.99)
print(high)

<center><img src="https://static01.nyt.com/images/2021/01/26/multimedia/26xp-photog/26xp-photog-superJumbo.jpg" width=70%></center>

Now let's apply our filters:

In [ ]:
cleaned = df[(df['price'] < high) & (df['price'] > low)]

And maybe we can filter on # bedrooms, too. And why not drop rows with missing data while we're at it.

In [ ]:
cleaned = cleaned[cleaned['bedrooms'] < 4].dropna()
cleaned.describe()

## 3.7 Continuous vs Categorical Data

In [ ]:
from matplotlib import pyplot as plt
fig, axarr = plt.subplots(1, 2, figsize=(13,5))
cleaned['subregion'].value_counts(sort=False).plot(kind='bar', ax=axarr[0], title='Sub-region')
axarr[0].set_ylabel("count")
cleaned['price'].plot(kind='kde', ax=axarr[1], title='Price ($)')

### 3.7.1 Binning your data
Sometimes you'll want to convert a continuous variable to categorical. pandas provides us with a few options for doing this:
- `pd.cut()`: evenly _spaced_ bins, or define your own breaks
- `pd.qcut()`: evenly _populated_ bins, or define your own percentile breaks

#### 3.7.1.1 `pd.cut()`

In [ ]:
pd.cut(cleaned['price'], 3, labels=['low', 'medium', 'high']).value_counts().plot(kind='bar')

In [ ]:
pd.cut(
    cleaned['price'], [0, 1000, 5000, 20000],
    labels=['low', 'medium', 'high']).value_counts(sort=False).plot(kind='bar')

#### 3.7.1.2 `pd.qcut()`

In [ ]:
quintile_df, bins = pd.qcut(
    cleaned['price'], 5,
    labels=['very low', 'low', 'average', 'high', 'very high'], retbins=True)

In [ ]:
pd.qcut(
    cleaned['price'], [0, .1, .3, .7, .9, 1],
    labels=['very low', 'low', 'medium', 'high', 'very high']).value_counts(sort=False).plot(kind='bar')

### 3.7.2 Dummy variables

Sometimes you might want to do the opposite: convert a categorical variable to a continuous or numeric variable. The way to do this is to create "dummy variables", where each category becomes its own _column_ with values that equal 1 if the _row_ belongs to the category and 0 otherwise:

In [ ]:
pd.get_dummies(cleaned['subregion']).sample(5)

You can then merge your dummy columns back onto the main dataframe

In [ ]:
cleaned.join(pd.get_dummies(cleaned['bedrooms'].astype(int), prefix='beds')).loc[:, 'bedrooms':].sample(5)

## 3.8 Putting it all together: Craigslist Rental Listings + SF1

Let's load another set of rental listings. This one is a dataset I've already cleaned for you. I've also geocoded the addresses to get lat/lon coordinates and Census Block IDs.

### 3.8.1 Merging data from two different sources

In [ ]:
rentals = pd.read_csv(
    'data/sfbay_geocoded.csv',
    usecols=['rent', 'bedrooms', 'sqft', 'fips_block', 'longitude', 'latitude'],
    dtype={'fips_block': str}  # load fips_block as str, numeric type will drop leading zero
)  
rentals.head()

And merge it with the census data using the FIPS block codes, which are named differently in the two DataFrames.

In [ ]:
rentals_sf1 = pd.merge(rentals, sf1, left_on='fips_block', right_on='blockfips')
rentals_sf1.head()

### 3.8.2 Multi-column group-by's

Now we can begin looking at this merged dataset.  Let's start by computing mean rents by county.

In [ ]:
county_rents = rentals_sf1.groupby(
    rentals_sf1['county_name'])[['rent']].mean().sort_values(by='rent', ascending=False)
county_rents.plot(kind='bar')

This result generally conforms to our expectations, with San Francisco having the highest average rent and Solano lowest. But what if Solano just has a higher percentage of 1 bedroom apartments? Could that bias our findings? How might we account for this possibility?

One way we could investigate the effect of total bedrooms is to include bedrooms as an additional segmentation variable:

In [ ]:
rentals_sf1.groupby(['county_name', 'bedrooms'])['rent'].mean().plot(kind='bar')

That bar chart is not ideal. Too small, and it would be nicer to make it separate colors for each number of bedrooms.  Also notice how the use of two groupby variables produces a MultiIndex, which makes for ugly axis labels at the very least. 

We can use `unstack()` to convert one of the indices from row values to columns:

In [ ]:
rentals_sf1.groupby(['county_name', 'bedrooms'])['rent'].mean().unstack()

Now we can plot a bar chart with the unstacked data, add a title, and set the figure size.

In [ ]:
rentals_sf1.groupby(
    ['county_name', 'bedrooms'])['rent'].mean().unstack().plot(
    kind='bar', figsize=(14,6), title='Average Rents by County and # Bedrooms', ylabel='rent ($)', xlabel='county')

Note that in one line of code we can filter, groupby, and plot our results.

**Question:** What's wrong with the plot above? Is it really showing us what we're interested in?

### 3.8.3 Pivot tables and Crosstabs

Remember when I said that in programming there is always more than one way to skin a cat? Here is another way of skinning this one that should look familiar to all of you excel power users out there:

In [ ]:
pd.pivot_table(rentals_sf1, values='rent', index=['bedrooms'], columns=['county_name']).plot(
    kind='bar', figsize=(14,6), title='Average Rents by County and Bedrooms')

The `pivot_table()` function makes it easy to add also can add partial totals, or "marginals", to our data:

In [ ]:
pd.pivot_table(rentals_sf1, values='rent', index='county_name', columns='bedrooms', margins=True)

Although mean is the default type of aggregation in pivot_table, you can use any aggregation method:

In [ ]:
pd.pivot_table(
    rentals_sf1, values='rent', index='county_name', columns='bedrooms', aggfunc="count", margins=True)

We could use the `count` aggregation method to get a full frequency distribution, a.k.a cross-tabulation

But then again, there's an even simpler way to do this in pandas

In [ ]:
pd.crosstab(rentals_sf1['county_name'], rentals_sf1['bedrooms'], margins=True)

And the `crosstab()` function comes with its own bells and whistles. For example, setting `normalize=True` will tells us the fraction of the region's total listings that are in each combination of county and number of bedrooms.

In [ ]:
pd.crosstab(rentals_sf1['county_name'], rentals_sf1['bedrooms'], margins=True, normalize=True)

We could also normalize just the rows (index) or the just the columns:

In [ ]:
pd.crosstab(rentals_sf1['county_name'], rentals_sf1['bedrooms'], margins=True, normalize='index')

In [ ]:
pd.crosstab(rentals_sf1['county_name'], rentals_sf1['bedrooms'], margins=True, normalize='columns')

What if we want to look at more statistics than just mean? We can combine several aggregation methods and compute them at one time.

In [ ]:
rentals_sf1[rentals_sf1['bedrooms'] < 4].groupby(
    ['county_name', 'bedrooms'])['rent'].agg(['mean', 'std', 'min', 'max']).reset_index().head()

### 3.8.4 Exploring correlations in your data

Pandas provides simple ways of computing correlation coefficients among the columns in your DataFrame.

In [ ]:
rentals_sf1[['rent', 'sqft']].corr()

And this method can be combined with groupby to compute correlation tables by group.

In [ ]:
rentals_sf1.groupby('county_name')[['rent', 'sqft']].corr()

### 3.8.5 Quantiles and User-defined Aggregation Functions

Below the `cut()` function to create categories for ranges of a variable. In this example we use 4 even intervals.

In [ ]:
sqft_cat = pd.cut(rentals_sf1['sqft'], 4)
sqft_cat.head()

Let's define our own aggregation function to get a standardized maximum rent for each sqft category. Standardization is the process of transforming your data such the the mean is 0 and the standard deviation is 1, which is accomplished by subtracting the mean and dividing by the standard deviation. By standardizing your data, you are able to make more generalized comparisons across groups.

In [ ]:
rentals_sf1.groupby(sqft_cat)['rent'].agg(
    max_rent='max', standardized_max=lambda x: (x.max() - x.mean()) / x.std())

So even though the smallest apartment size category has the lowest maximum rent, it is the most _extreme_ maximum rent relative to its group mean!

## 3.9 Exercises

Try practicing these techniques on your own, to do the following:

* Calculate the mean sqft of rental listings by county
* Calculate the standard deviation (std) of sqft of rental listings by county and bedroom
* Add a new column with a normalized sqft, substracting the mean sqft by bedroom from each listing's sqft 
* Compute correlation coefficients among rent, sqft, pct_white, pct_black, pct_asian and pct_hisp, by county and for the region
* Redo the statistical profile on rents by categories of sqft range using 10 quantiles rather than 4 equal bins


# 4. For Next Time
- Continue work on HW2

# 5 Questions?